# Gamelytics: Mobile Analytics Challenge
* Retention Analysis
    🔍 Objective: Create a flexible retention calculator that specifies a dataframe, start date, and a period.


#To Do:
* A/B Testing for Promotional Offers
    🔍 Objective: Identify the best-performing promotional offer set by comparing key revenue metrics.

* Event Performance Evaluation in "Plants & Gardens"
    🔍 Objective: Develop metrics to assess the success of a time-limited in-game event where players can earn unique rewards.


: 

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import datetime

#Import data
path = kagglehub.dataset_download("debs2x/gamelytics-mobile-analytics-challenge")

reg_data = pd.read_csv(path+'/reg_data.csv',sep=';')
auth_data = pd.read_csv(path+'/auth_data.csv',sep=';')
ab_test = pd.read_csv(path+'/ab_test.csv',sep=';')


: 

### Flexible Retention Calculation 

In [4]:
#merge dfs
df = pd.merge(auth_data, reg_data, on='uid', how='left')
df['auth_dt'] = pd.to_datetime(df['auth_ts'], unit='s')
df['reg_dt'] = pd.to_datetime(df['reg_ts'], unit='s')

In [22]:
print(df.columns, df['reg_dt'].min(),
df['reg_dt'].max())
df.head()

Index(['auth_ts', 'uid', 'reg_ts', 'auth_dt', 'reg_dt'], dtype='object') 1998-11-18 09:43:43 2020-09-23 15:17:24


,auth_ts,uid,reg_ts,auth_dt,reg_dt
0,911382223,1,911382223,1998-11-18 09:43:43,1998-11-18 09:43:43
1,932683089,2,932683089,1999-07-22 22:38:09,1999-07-22 22:38:09
2,932921206,2,932683089,1999-07-25 16:46:46,1999-07-22 22:38:09
3,933393015,2,932683089,1999-07-31 03:50:15,1999-07-22 22:38:09
4,933875379,2,932683089,1999-08-05 17:49:39,1999-07-22 22:38:09


In [23]:
#Rolling retention
def retention_calc_dt(df, start_date, period):
    start_date = pd.to_datetime(start_date)
    allowed = {
        pd.Timedelta(days=7),
        pd.Timedelta(days=14),
        pd.Timedelta(days=30),
    }
    
    # Validate start_date
    min_date = pd.to_datetime("1999-08-23")
    max_date = pd.to_datetime("2020-08-21")
    
    start_date_dt = pd.to_datetime(start_date, errors="coerce")
    
    if pd.isna(start_date_dt):
        raise ValueError(f"start_date must be a valid date. Got: {start_date}")
    
    if start_date_dt < min_date or start_date_dt > max_date:
        raise ValueError(f"start_date must be between {min_date.strftime('%Y-%m-%d')} and {max_date.strftime('%Y-%m-%d')}. Got: {start_date_dt.strftime('%Y-%m-%d')}")

    if isinstance(period, (int, float)):
        period_td = pd.Timedelta(days=int(period))
    else:
        period_td = pd.to_timedelta(period)

    if period_td not in allowed:
        allowed_str = ", ".join(f"{td.days} days" for td in sorted(allowed))
        raise ValueError(f"period must be one of: {allowed_str}. Got: {period_td}")
    
     # Define time windows
    period_start = start_date_dt
    period_end = start_date_dt + period_td
    pre_period_start = start_date_dt - period_td
   
    # start: distinct users who logged in during the period before start_date
    users_before_period = df[
        (df['auth_dt'] >= pre_period_start) & 
        (df['auth_dt'] < start_date_dt)
    ]["uid"].unique()
    start = len(users_before_period)


    # end: distinct users who logged in during the period (from start_date to start_date + period)
    users_during_period = df[
        (df['auth_dt'] >= period_start) & 
        (df['auth_dt'] < period_end)
    ]["uid"].unique()
    end = len(users_during_period)
    
    users_before_start = df[df['auth_dt'] < start_date_dt]["uid"].unique()
    new_users = set(users_during_period) - set(users_before_start)
    new = len(new_users)

    if start == 0:
        retention_rate = 0.0
    else:
        retention_rate = (end-new) / start
    
    # statement = print(f"{retention_rate}% of users were retained over the {period} day period from {start_date} to {start_date + period}")
    statement = (
        f"{retention_rate * 100:.2f}% of users were retained over the {period_td.days}-day period "
        f"from {period_start:%Y-%m-%d} to {period_end:%Y-%m-%d}.")    
    print(statement)
    return statement
    
    

In [7]:
retention_calc_dt(df, '2000-07-22', 30)
retention_calc_dt(df, '2001-07-22', 30)
retention_calc_dt(df, '2002-07-22', 30)
retention_calc_dt(df, '2003-07-22', 30)
retention_calc_dt(df, '2004-07-22', 30)


100.00% of users were retained over the 30-day period from 2000-07-22 to 2000-08-21.
100.00% of users were retained over the 30-day period from 2001-07-22 to 2001-08-21.
66.67% of users were retained over the 30-day period from 2002-07-22 to 2002-08-21.
25.00% of users were retained over the 30-day period from 2003-07-22 to 2003-08-21.
40.00% of users were retained over the 30-day period from 2004-07-22 to 2004-08-21.


'40.00% of users were retained over the 30-day period from 2004-07-22 to 2004-08-21.'

In [8]:
#Cohort retention
#get users who belong in a specific cohort (registered in January 1999, for instance)
#see if they logged in within some period (e.g. 7 days) after registering
def cohort_retention_simple(df, cohort_month, retention_days): #cohort_month should be a tuple of year and month (12,1999)
    """
    Simple and efficient cohort retention
    """
    year, month = cohort_month
    
    # 1. Define cohort period
    cohort_start = pd.Timestamp(year=year, month=month, day=1)
    if month == 12:
        cohort_end = pd.Timestamp(year=year+1, month=1, day=1)
    else:
        cohort_end = pd.Timestamp(year=year, month=month+1, day=1)
    
    # 2. Get cohort: users who registered in this month
    cohort_registrations = df[
        (df['reg_dt'] >= cohort_start) & 
        (df['reg_dt'] < cohort_end)
    ][['uid', 'reg_dt']].drop_duplicates(subset='uid')
    
    total_cohort_size = len(cohort_registrations)
    
    if total_cohort_size == 0:
        return {day: 0.0 for day in retention_days}
    
    # 3. For each retention period, check logins
    retention_rates = {}
    
    for days in retention_days:
        # Create cutoff dates for each user
        cohort_registrations['cutoff'] = cohort_registrations['reg_dt'] + pd.Timedelta(days=days)
        
        # Check which users have at least one login before their cutoff
        retained_users = []
        for _, user_row in cohort_registrations.iterrows():
            uid = user_row['uid']
            reg_date = user_row['reg_dt']
            cutoff = user_row['cutoff']
            
            # Check if user logged in between reg_date and cutoff
            has_login = df[
                (df['uid'] == uid) &
                (df['auth_dt'] > reg_date) &  # After registration (not same-day)
                (df['auth_dt'] <= cutoff)
            ].shape[0] > 0
            
            retained_users.append(has_login)
        
        retention_rates[days] = sum(retained_users) / total_cohort_size
    
    return retention_rates, total_cohort_size

In [9]:
cohort_retention_simple(df, (2000,12), [7,14,30])

({7: 0.0, 14: 0.0, 30: 0.0}, 1)

In [10]:
def cohort_retention_simple(df, cohort_month, retention_days):
    """
    Simple and efficient cohort retention
    
    Parameters:
    - df: dataframe with 'reg_dt' and 'auth_dt' columns
    - cohort_month: tuple (year, month) e.g., (2000, 12) OR list of tuples [(2000,12), (2001,1), (2002,2)]
    - retention_days: list of days to check [7, 14, 30]
    
    Returns:
    - If single cohort: dict of retention rates and cohort size
    - If multiple cohorts: dict with cohort labels as keys, each containing retention rates and size
    """
    # Handle both single cohort and list of cohorts
    if isinstance(cohort_month, tuple):
        cohort_months = [cohort_month]
        single_cohort = True
    elif isinstance(cohort_month, list):
        cohort_months = cohort_month
        single_cohort = False
    else:
        raise ValueError(f"cohort_month must be tuple (year, month) or list of tuples. Got: {type(cohort_month)}")
    
    results = {}
    
    for year, month in cohort_months:
        # 1. Define cohort period
        cohort_start = pd.Timestamp(year=year, month=month, day=1)
        if month == 12:
            cohort_end = pd.Timestamp(year=year+1, month=1, day=1)
        else:
            cohort_end = pd.Timestamp(year=year, month=month+1, day=1)
        
        # 2. Get cohort: users who registered in this month
        cohort_registrations = df[
            (df['reg_dt'] >= cohort_start) & 
            (df['reg_dt'] < cohort_end)
        ][['uid', 'reg_dt']].drop_duplicates(subset='uid')
        
        total_cohort_size = len(cohort_registrations)
        
        if total_cohort_size == 0:
            retention_rates = {day: 0.0 for day in retention_days}
        else:
            # 3. For each retention period, check logins
            retention_rates = {}
            
            for days in retention_days:
                # Create cutoff dates for each user
                cohort_registrations['cutoff'] = cohort_registrations['reg_dt'] + pd.Timedelta(days=days)
                
                # Check which users have at least one login before their cutoff
                retained_users = []
                for _, user_row in cohort_registrations.iterrows():
                    uid = user_row['uid']
                    reg_date = user_row['reg_dt']
                    cutoff = user_row['cutoff']
                    
                    # Check if user logged in between reg_date and cutoff
                    has_login = df[
                        (df['uid'] == uid) &
                        (df['auth_dt'] > reg_date) &  # After registration (not same-day)
                        (df['auth_dt'] <= cutoff)
                    ].shape[0] > 0
                    
                    retained_users.append(has_login)
                
                retention_rates[days] = sum(retained_users) / total_cohort_size
        
        # Store results with cohort label
        cohort_label = f"{year}-{month:02d}"
        results[cohort_label] = {
            'retention_rates': retention_rates,
            'cohort_size': total_cohort_size
        }
    
    # Return format depends on input
    if single_cohort:
        # Return same format as before for backward compatibility
        first_result = list(results.values())[0]
        return first_result['retention_rates'], first_result['cohort_size']
    else:
        # Return dict with all cohorts
        return results

In [11]:
def cohort_retention_batch(df, cohort_months, retention_days):
    """
    Process multiple cohorts efficiently in one pass
    """
    # Pre-filter auth data
    auth_after_reg = df[df['auth_dt'] > df['reg_dt']].copy()
    
    # Create a mapping of uid to registration date (for all users)
    uid_to_reg = df[['uid', 'reg_dt']].drop_duplicates(subset='uid').set_index('uid')['reg_dt']
    
    results = {}
    
    for year, month in cohort_months:
        # Define cohort period
        cohort_start = pd.Timestamp(year=year, month=month, day=1)
        if month == 12:
            cohort_end = pd.Timestamp(year=year+1, month=1, day=1)
        else:
            cohort_end = pd.Timestamp(year=year, month=month+1, day=1)
        
        # Get cohort users
        cohort_mask = (df['reg_dt'] >= cohort_start) & (df['reg_dt'] < cohort_end)
        cohort_uids = set(df[cohort_mask]['uid'].unique())
        total_cohort_size = len(cohort_uids)
        
        if total_cohort_size == 0:
            retention_rates = {day: 0.0 for day in retention_days}
        else:
            # Get auth events for this cohort
            cohort_auths = auth_after_reg[auth_after_reg['uid'].isin(cohort_uids)].copy()
            
            # Add registration dates
            cohort_auths['reg_dt'] = cohort_auths['uid'].map(uid_to_reg)
            
            # Calculate days since registration
            cohort_auths['days_since_reg'] = (cohort_auths['auth_dt'] - cohort_auths['reg_dt']).dt.days
            
            # Calculate retention for each period
            retention_rates = {}
            for days in retention_days:
                users_with_login = cohort_auths[
                    cohort_auths['days_since_reg'] <= days
                ]['uid'].unique()
                retention_rates[days] = len(users_with_login) / total_cohort_size
        
        results[f"{year}-{month:02d}"] = {
            'retention_rates': retention_rates,
            'cohort_size': total_cohort_size
        }
    
    return results

In [12]:
# Generate all cohort months
cohort_months = [(year, month) for year in range(1999, 2020) for month in range(1, 13)]

# Process all at once (much faster!)
results = cohort_retention_batch(df, cohort_months, [7, 14, 30])

# Print results
for cohort_label, data in results.items():
    if data['cohort_size'] > 0:  # Only print non-empty cohorts
        print(f"{cohort_label}: size={data['cohort_size']}, retention={data['retention_rates']}")

1999-07: size=1, retention={7: 1.0, 14: 1.0, 30: 1.0}
2000-01: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2000-05: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2000-09: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2000-12: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-03: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-05: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-07: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-09: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-11: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2001-12: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-02: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-03: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-05: size=1, retention={7: 1.0, 14: 1.0, 30: 1.0}
2002-06: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-07: size=1, retention={7: 1.0, 14: 1.0, 30: 1.0}
2002-08: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-09: size=1, retention={7: 0.0, 14: 0.0, 30: 0.0}
2002-10: size=1, retention={

In [17]:
import plotly.graph_objects as go
import plotly.express as px

def plot_cohort_retention_plotly(results, retention_days=[7, 14, 30], min_cohort_size=0):
    """
    Interactive plot of cohort retention rates over time using Plotly
    
    Parameters:
    - results: output from cohort_retention_batch function
    - retention_days: list of retention periods to plot
    - min_cohort_size: minimum cohort size to include (default: 0, include all)
    """
    # Transform results into a DataFrame
    plot_data = []
    
    for cohort_label, data in results.items():
        # Filter by minimum cohort size
        if data['cohort_size'] < min_cohort_size:
            continue
            
        year, month = map(int, cohort_label.split('-'))
        cohort_date = pd.Timestamp(year=year, month=month, day=1)
        
        for days in retention_days:
            if days in data['retention_rates']:
                plot_data.append({
                    'cohort_date': cohort_date,
                    'retention_days': f'{days} days',
                    'retention_rate': data['retention_rates'][days] * 100,  # Convert to percentage
                    'cohort_size': data['cohort_size']
                })
    
    df_plot = pd.DataFrame(plot_data)
    
    if len(df_plot) == 0:
        print("No data to plot")
        return None
    
    # Sort by date for proper line plotting
    df_plot = df_plot.sort_values('cohort_date')
    
    # Create interactive plot
    fig = px.line(df_plot, 
                  x='cohort_date', 
                  y='retention_rate',
                  color='retention_days',
                  markers=True,
                  title='Cohort Retention Rates Over Time',
                  labels={
                      'cohort_date': 'Cohort Month',
                      'retention_rate': 'Retention Rate (%)',
                      'retention_days': 'Retention Period'
                  },
                  hover_data=['cohort_size'])
    
    # Customize layout
    fig.update_layout(
        xaxis_title='Cohort Month',
        yaxis_title='Retention Rate (%)',
        hovermode='x unified',
        width=1200,
        height=600,
        legend=dict(
            title='Retention Period',
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        ),
        template='plotly_white'
    )
    
    # Format x-axis to show dates nicely
    fig.update_xaxes(
        tickformat='%Y-%m',
        dtick='M3'  # Show every 3 months
    )
    
    # Format y-axis to show percentage
    fig.update_yaxes(
        ticksuffix='%'
    )
    
    fig.show()
    return fig

In [18]:
# Generate all cohort months
cohort_months = [(year, month) for year in range(1999, 2020) for month in range(1, 13)]

# Get results
results = cohort_retention_batch(df, cohort_months, [7, 14, 30])

# Plot with Plotly
plot_cohort_retention_plotly(results, retention_days=[7, 14, 30])

# Optional: Filter out small cohorts (e.g., only show cohorts with at least 10 users)
plot_cohort_retention_plotly(results, retention_days=[7, 14, 30], min_cohort_size=10)

In [19]:
# Quick check: Look at a sample cohort's data
year, month = 2000, 1
cohort_start = pd.Timestamp(year=year, month=month, day=1)
cohort_end = pd.Timestamp(year=year, month=month+1, day=1)

cohort_users = df[(df['reg_dt'] >= cohort_start) & (df['reg_dt'] < cohort_end)]['uid'].unique()[:5]

# Check a few users' login patterns
for uid in cohort_users[:3]:
    user_data = df[df['uid'] == uid].sort_values('auth_dt')
    reg_date = user_data['reg_dt'].iloc[0]
    print(f"\nUser {uid}:")
    print(f"  Registered: {reg_date}")
    for _, row in user_data.iterrows():
        days_after = (row['auth_dt'] - reg_date).days
        print(f"  Login: {row['auth_dt']} ({days_after} days after registration)")



User 3:
  Registered: 2000-01-13 22:27:27
  Login: 2000-01-13 22:27:27 (0 days after registration)


In [20]:

tmp = df.dropna(subset=["auth_dt", "reg_dt"]).copy()
tmp = tmp[tmp["auth_dt"] >= tmp["reg_dt"]]
tmp["days_since_reg"] = (tmp["auth_dt"] - tmp["reg_dt"]).dt.total_seconds() / 86400

tmp["days_since_reg"].describe(percentiles=[.5, .9, .95, .99])


count    9.601013e+06
mean     4.781572e+02
std      5.811977e+02
min      0.000000e+00
50%      2.786050e+02
90%      1.246012e+03
95%      1.661160e+03
99%      2.585477e+03
max      7.728532e+03
Name: days_since_reg, dtype: float64

In [21]:
user_span = (
    tmp.groupby("uid")["days_since_reg"]
       .max()
       .sort_values(ascending=False)
       .head(20)
)
user_span


uid
2      7728.532176
108    5589.764606
158    5353.820417
176    5295.526956
191    5253.105046
204    5211.774641
212    5190.943171
224    5158.316273
280    5026.504398
287    5009.960567
336    4917.165359
390    4828.150949
467    4728.799363
483    4703.896632
555    4619.319514
562    4609.682894
584    4585.118750
600    4573.159421
613    4559.327199
623    4547.187465
Name: days_since_reg, dtype: float64